# VLM Cross-Modal Deep Hashing — Demo Dashboard

CPU-only notebook for viewing training results, running inference, and searching.
Reads metrics from Google Drive — no GPU required.

In [ ]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

import os
DRIVE_BASE = "/content/drive/MyDrive/vlm_quantization"
assert os.path.isdir(DRIVE_BASE), f"Drive directory not found: {DRIVE_BASE}\nRun the training notebook first."
print(f"Drive base: {DRIVE_BASE}")

In [ ]:
# Cell 2: Clone Repo + Install Dependencies + Load .env
!git clone https://github.com/hyunlord/vlm_quantization.git /content/vlm_quantization 2>/dev/null || true
%cd /content/vlm_quantization
!git pull
!pip install -q -r requirements.txt
!pip install -q pyngrok

import os
env_path = "/content/drive/MyDrive/vlm_quantization/.env"
if os.path.exists(env_path):
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith("#") and "=" in line:
                key, val = line.split("=", 1)
                os.environ[key.strip()] = val.strip()
    print(".env loaded from Google Drive")
else:
    print(f"No .env found at {env_path}")

In [ ]:
# Cell 3: Build Frontend (static export)
!curl -fsSL https://deb.nodesource.com/setup_20.x | bash - > /dev/null 2>&1
!apt-get -qq install -y nodejs > /dev/null 2>&1
!node --version
!cd /content/vlm_quantization/monitor/frontend && npm install --silent && npm run build
print("Frontend built \u2192 monitor/frontend/out/")

In [ ]:
# Cell 4: Start Dashboard Server + ngrok Tunnel
import os
import threading
import time

import uvicorn
from pyngrok import ngrok

# Point metrics DB to Google Drive (read historical training data)
MONITOR_DIR = "/content/drive/MyDrive/vlm_quantization/monitor"
os.makedirs(MONITOR_DIR, exist_ok=True)
os.environ["MONITOR_DB_PATH"] = f"{MONITOR_DIR}/metrics.db"

# Check for existing training data
db_path = os.environ["MONITOR_DB_PATH"]
if os.path.isfile(db_path):
    db_size = os.path.getsize(db_path) / 1024
    print(f"Found metrics DB: {db_path} ({db_size:.0f} KB)")
else:
    print(f"No metrics DB found at {db_path}")
    print("Run the training notebook first to generate training data.")

# Check for checkpoints (needed for inference/search)
ckpt_dir = "/content/drive/MyDrive/vlm_quantization/checkpoints"
if os.path.isdir(ckpt_dir):
    ckpts = [f for f in os.listdir(ckpt_dir) if f.endswith(".ckpt")]
    print(f"Checkpoints: {len(ckpts)} found in {ckpt_dir}")
else:
    print(f"No checkpoints directory at {ckpt_dir}")

token = os.environ.get("NGROK_AUTH_TOKEN", "")
if token:
    ngrok.set_auth_token(token)

def run_server():
    uvicorn.run("monitor.server.app:app", host="0.0.0.0", port=8000, log_level="warning")

threading.Thread(target=run_server, daemon=True).start()
time.sleep(3)

try:
    tunnel = ngrok.connect(8000)
    print(f"\n Dashboard: {tunnel.public_url}")
    print(f" Metrics DB: {db_path}")
    print(f" Checkpoint dir: {ckpt_dir}\n")
except Exception as e:
    err_msg = str(e)
    if "ERR_NGROK_334" in err_msg or "endpoint already online" in err_msg.lower():
        print("ERROR: ngrok endpoint already in use by another notebook.")
        print("  → Stop the training notebook's ngrok tunnel first,")
        print("    then re-run this cell.")
        print(f"\n  (Server is running locally on port 8000 without tunnel)")
    else:
        raise